In [5]:
% load_ext autoreload
% autoreload 2
% reload_ext autoreload

UsageError: Line magic function `%` not found.


In [13]:
import os
import joblib
import tensorflow as tf
import pandas as pd

from src.features.data_engineering import preprocess_prediction_data

In [14]:
rolling_columns = ['assists', 'bonus', 'bps', 'clean_sheets',
                   'creativity', 'goals_conceded', 'goals_scored',
                   'ict_index', 'influence', 'minutes',
                   'saves', 'selected', 'player_team_score', 'opponent_team_score', 'threat',
                   'total_points', 'transfers_in', 'transfers_out',
                   'value', 'yellow_cards']

times = ['all', 6, 3]

In [15]:
x_prediction, x_target = preprocess_prediction_data('2021-22', 30, rolling_columns=rolling_columns, rolling_times=times, opponent_team_stats=True)

start scraping missing teams data season:  2021-22
End of scraping
Are there any NaN values? - False


In [16]:
x_target.head(10)

,name,element,team,GW,season,opponent_next_gameweek
17621,Owen Dodgson,565,Burnley,29,2021-22,Manchester City
17621,Owen Dodgson,565,Burnley,29,2021-22,Everton
17621,Owen Dodgson,565,Burnley,29,2021-22,Norwich
17621,Owen Dodgson,565,Burnley,29,2021-22,West Ham
17621,Owen Dodgson,565,Burnley,29,2021-22,Southampton
17621,Owen Dodgson,565,Burnley,29,2021-22,Wolverhampton Wanderers
17598,Sam Waller,708,Burnley,29,2021-22,Manchester City
17598,Sam Waller,708,Burnley,29,2021-22,Everton
17598,Sam Waller,708,Burnley,29,2021-22,Norwich
17598,Sam Waller,708,Burnley,29,2021-22,West Ham


In [17]:
# load 'model_mlp_rolling_opponent_stats_features' model
root_dir = os.path.dirname(os.path.abspath(''))
model = tf.keras.models.load_model(root_dir + '/models/baseline/model_mlp_rolling_opponent_stats_features.h5')
x_scaler = joblib.load(root_dir + '/models/baseline/SCALER-model_mlp_rolling_opponent_stats_features.gz')

In [18]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 128)               12544     
                                                                 
 dense_1 (Dense)             (None, 256)               33024     
                                                                 
 dense_2 (Dense)             (None, 126)               32382     
                                                                 
 dense_3 (Dense)             (None, 64)                8128      
                                                                 
 dense_4 (Dense)             (None, 1)                 65        
                                                                 
Total params: 86,143
Trainable params: 86,143
Non-trainable params: 0
_________________________________________________________________


In [19]:
y_pred = pd.Series(model.predict(x_scaler.transform(x_prediction)).reshape(-1, ), index=x_prediction.index, name='predicted_total_points_next_gameweek')
prediction_df = pd.concat([y_pred, x_target], axis=1)

In [20]:
prediction_df

,predicted_total_points_next_gameweek,name,element,team,GW,season,opponent_next_gameweek
17621,0.000961,Owen Dodgson,565,Burnley,29,2021-22,Manchester City
17621,0.037667,Owen Dodgson,565,Burnley,29,2021-22,Everton
17621,0.051699,Owen Dodgson,565,Burnley,29,2021-22,Norwich
17621,0.004169,Owen Dodgson,565,Burnley,29,2021-22,West Ham
17621,0.013536,Owen Dodgson,565,Burnley,29,2021-22,Southampton
...,...,...,...,...,...,...,...
18728,0.217791,Alphonse Areola,489,West Ham,30,2021-22,Chelsea
18674,0.044222,Jamal Baptiste,554,West Ham,30,2021-22,Everton
18674,0.026542,Jamal Baptiste,554,West Ham,30,2021-22,Brentford
18674,0.056511,Jamal Baptiste,554,West Ham,30,2021-22,Burnley


In [44]:
# group prediction_df by 'name', 'element', 'team', 'GW", 'season', sum up 'predicted_total_points_next_gameweek' and make 'opponent_next_gameweek' items a list
prediction_df_sum = prediction_df.groupby(['name', 'element', 'team', 'GW', 'season']).sum().reset_index()
list_of_opponents = prediction_df.groupby(['name', 'element', 'team', 'GW', 'season'])['opponent_next_gameweek'].apply(list).reset_index()['opponent_next_gameweek']

prediction_df_sum['next_4_GWs_matches'] = list_of_opponents.apply(lambda x: len(x))
prediction_df_sum['opponent_next_gameweek'] = list_of_opponents

In [45]:
prediction_df_sum

,name,element,team,GW,season,predicted_total_points_next_gameweek,next_4_GWs_matches,opponent_next_gameweek
0,Aaron Connolly,72,Brighton,29,2021-22,0.385969,5,"[Norwich, Arsenal, Tottenham, Manchester City,..."
1,Aaron Cresswell,411,West Ham,30,2021-22,13.169465,4,"[Everton, Brentford, Burnley, Chelsea]"
2,Aaron Lennon,575,Burnley,29,2021-22,14.788957,6,"[Manchester City, Everton, Norwich, West Ham, ..."
3,Aaron Ramsdale,559,Arsenal,30,2021-22,10.169611,5,"[Crystal Palace, Brighton, Southampton, Chelse..."
4,Aaron Ramsey,496,Aston Villa,30,2021-22,-0.048319,3,"[Wolverhampton Wanderers, Tottenham, Leicester]"
...,...,...,...,...,...,...,...,...
708,Zach Awe,705,Arsenal,30,2021-22,-0.181945,5,"[Crystal Palace, Brighton, Southampton, Chelse..."
709,Zack Steffen,260,Manchester City,29,2021-22,0.340187,4,"[Burnley, Liverpool, Brighton, Watford]"
710,Zak Swanson,711,Arsenal,30,2021-22,-0.187515,5,"[Crystal Palace, Brighton, Southampton, Chelse..."
711,Álvaro Fernández,556,Brentford,30,2021-22,0.305082,4,"[Chelsea, West Ham, Watford, Tottenham]"
